In [8]:
import pandas as pd

## Reading a saved google trends csv to a dataframe

- csvpath variable should be changed

In [13]:
csvpath = "/home/eduardotc/Downloads/multiTimeline(2).csv"
df = pd.read_csv(csvpath, header=1)
df
 

,Mês,plasmonic: (Mundo)
0,2004-01,0
1,2004-02,0
2,2004-03,0
3,2004-04,33
4,2004-05,0
...,...,...
233,2023-06,21
234,2023-07,16
235,2023-08,17
236,2023-09,18


## Sum of a google trends date column by year

- df DataFrame should be defined

- Considers the date column is a string, not a timestamp, which happens when
importing a downloaded from browser csv

In [10]:
# Visual Python: Data Analysis > File
vp_df = pd.read_csv('../../../csv/plasmonic_scopus.csv', index_col=0)
vp_df

,date,articles
0,2000-01-01,0
1,2000-02-01,0
2,2000-03-01,0
3,2000-04-01,1
4,2000-06-01,0
...,...,...
259,2023-08-01,895
260,2023-09-01,722
261,2023-10-01,559
262,2023-11-01,321


In [11]:
df_names = df.columns.values.tolist()
col1 = df_names[0]
col2 = df_names[1]
result_df = df.groupby(df[col1].str[:4])[col2].sum().reset_index()
print(result_df)

     Mês  plasmonic: (Mundo)
0   2004                  33
1   2005                 164
2   2006                 148
3   2007                  94
4   2008                 128
5   2009                 183
6   2010                 190
7   2011                 259
8   2012                 315
9   2013                 324
10  2014                 335
11  2015                 287
12  2016                 297
13  2017                 262
14  2018                 251
15  2019                 217
16  2020                 200
17  2021                 172
18  2022                 224
19  2023                 188


## Querying a keyword all time google trends dataframe

- keyword should be defined

### Trends filters

- cat → Category to narrow results (integer)

    - Science: 174
        - Chemistry: 505
        - Biological sciences: 440
        - Computer science: 1227
        - Ecology & Environment: 442
        - Engineering & Technology: 231
        - Mathematics: 436
        - Physics: 444
        - Scientific Equipment: 445
        - Scientific Institutions: 446
    - Health: 45
        - Health conditions: 419
        - Pharmacy: 248
        - Drugs & medications: 646
        - Health Foundations & Medical Research: 252

- geo → Two letter country abreviation

    - Defaults to world
    - Example US

- timeframe

    - Date to start from
    - Defaults to last 5yrs, 'today 5-y'.
    - Everything 'all'
    - Specific dates, 'YYYY-MM-DD YYYY-MM-DD' example '2016-12-14 2017-01-25'
    - Specific datetimes, 'YYYY-MM-DDTHH YYYY-MM-DDTHH' example '2017-02-06T10 2017-02-12T07'

- gprop

    - Can be images, news, youtube or froogle (for Google Shopping results)

In [12]:
from pytrends.request import TrendReq
keyword = "plasmonic"
save_path = f"/home/eduardotc/Programação/csv/{keyword}.csv"
pytrends = TrendReq(hl='en-US', tz=360)
kw_list = [keyword]
pytrends.build_payload(kw_list, cat=0, timeframe='all', geo='', gprop='')
df = pytrends.interest_over_time()
df = df.reset_index()
df = df.drop('isPartial', axis=1)
print(df)
save_inp = str(input("Should save the dataframe to a csv? (y/n) "))
if save_inp == 'y':
    df.to_csv(path_or_buf=save_path)

ModuleNotFoundError: No module named 'pytrends'

## Trends queryed dataframe treatment

Sum of the dataframe by same year time stamp from the first column

Statistics by pandas agg

In [ ]:
df_names = df.columns.values.tolist()
col1 = df_names[0]
col2 = df_names[1]
df.groupby(df['date'].dt.year)[col2].agg(['sum', 'mean', 'max'])

# Querying pubmed articles per year-month

- Variables ystart and yend defines first and last year to analyze

- keyword to search number of published articles defined from variable

In [ ]:
from metapub import PubMedFetcher
fetch = PubMedFetcher()
from time import sleep
import numpy as np
import pandas as pd
keyword = 'singlet oxygen'
save_path = f'/home/eduardotc/Programação/csv/{keyword}_pubmed.csv'
ystart = 1990
yend = 2023
df = pd.DataFrame({'date': [],
       'articles': []})
month_list = np.arange(1, 13, 1)
months_days = pd.DataFrame({'month': month_list,
                            'ends': [31, 28, 31, 30, 31, 30, 31, 31, 30, 31,
                                     30, 31]})
year_list = np.arange(ystart, yend+1, 1)
for year in year_list:
    for month in months_days['month']:
        endsin = (months_days.loc[months_days['month'] == month, 'ends'])[month-1]
        pmids = fetch.pmids_for_query(f'{keyword} '+str(year)+f'/{month}/01[MDAT] : '+str(year)+f'/{month}/{endsin}[MDAT]',retmax=100000000)
        df.loc[len(df)] = [f'{year}-{month}', len(pmids)]
        print(f"{year}-{month}: ", len(pmids))
        sleep(0.5)
pd.to_datetime(df.date, format="%Y-%m")
df
save_inp = str(input("Should save the dataframe to a csv? (y/n) "))
if save_inp == 'y':
    df.to_csv(path_or_buf=save_path)

# Reading and analysis of pubmed csv

- Path to the saved pubmed dataframe defined in variables

- Grouped articles by year

- Statistics generated by pandas agg

In [ ]:
import pandas as pd
csvpath = '/home/eduardotc/Programação/csv/plasmonic_pubmed.csv'
df = pd.read_csv(csvpath, index_col=0)
df['date'] = pd.to_datetime(df.date, format="%Y-%m")
df = df.groupby(df['date'].dt.year)['articles'].agg(['sum', 'mean', 'max'])
df

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

# Matplotlib bar plotting

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
fig, ax = plt.subplots(figsize=(12, 4), dpi=400)
plt.xlabel('Year')
plt.ylabel('PubMed Articles')
plt.xlim(2000, 2023)
ax.set_title("Plasmonic")
ax.bar(df.index.values, df['sum'], label=df['sum'])
plt.savefig('/home/eduardotc/Programação/tstbar.png')

# Scopus keyword per year-month query

- Keyword defined in variables

- Years defined in variables

In [ ]:
from pybliometrics.scopus import ScopusSearch
import numpy as np
import pandas as pd
keyword = 'photodynamic therapy'
save_path = f'/home/eduardotc/Programação/csv/{keyword}_scopus.csv'
months_list = ['january', 'february', 'march', 'april', 'june', 'july',
               'august', 'september', 'october', 'november', 'december']
df = pd.DataFrame({'date': [],
                   'articles': []})
ystart = 2000
yend = 2023
years_list = np.arange(ystart, yend+1, 1)
for year in years_list:
    for month in months_list:
        s = ScopusSearch(f'KEY {keyword}, PUBDATETXT({month} {year})', subscriber=False)
        print(f"{year}-{month}: ", s.get_results_size())
        df.loc[len(df)] = [f'{year}-{month}', s.get_results_size()]
        sleep(9)
df['date'] = pd.to_datetime(df.date.astype(str), format="%Y-%B")
save_inp = str(input("Should save the dataframe to a csv? (y/n) "))
if save_inp == 'y':
    df.to_csv(path_or_buf=save_path)

## Converting and analysing scopus dataframe

- Direct from above script, without reading csv

In [ ]:
import pandas as pd
df['date'] = pd.to_datetime(df.date.astype(str), format="%Y-%B")
df = df.groupby(df['date'].dt.year)['articles'].agg(['sum', 'mean', 'max'])
print(df)

# Matplotlib default configs

In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler

plt.rcParams["pdf.use14corefonts"] = True
# trigger core fonts for PS backend
plt.rcParams["ps.useafm"] = True
# plt.rcParams['backend'] = 'Agg'
plt.rcParams.update(
    {
        "figure.dpi": 100,
        "font.size": 18,
        "figure.facecolor": "white",
        "figure.figsize": [10, 6],
        "figure.frameon": True,
        "figure.titlesize": "large",
        "figure.titleweight": "bold",
        "figure.labelsize": "medium",
        "figure.labelweight": "normal",
        "figure.edgecolor": "#000000",
    }
)

plt.rcParams.update(
    {
        "font.style": "normal",
        "font.weight": "bold",
        "font.family": "QTHelvet-Black",
        "font.sans-serif": "QTHelvet-Black",
        "font.size": 18,
    }
)

plt.rcParams.update(
    {
        "axes.grid": True,
        "axes.grid.axis": "both",
        "axes.grid.which": "major",
        "axes.labelcolor": "#172038",
        "axes.edgecolor": "#bcbcbc",
        "axes.facecolor": "#eeeeee",
        "axes.linewidth": 1.0,
    }
)

plt.rcParams.update(
    {
        "grid.alpha": 1.0,
        "grid.color": "#b2b2b2",
        "grid.linestyle": "--",
        "grid.linewidth": 0.5,
        "grid.alpha": 1.0,
    }
)

plt.rcParams.update(
    {
        "xtick.labelsize": "medium",
        "xtick.major.pad": 3.5,
        "xtick.major.size": 3.5,
        "xtick.alignment": "center",
        "xtick.color": "#000000",
        "ytick.labelsize": "x-small",
        "ytick.major.pad": 3.5,
        "ytick.major.size": 3.5,
        "ytick.alignment": "center_baseline",
        "ytick.color": "#000000",
    }
)
plt.rcParams.update(
    {
        "axes.prop_cycle": cycler(
            "color",
            [
                "#7499ff",
                "#348ABD",
                "#988ED5",
                "#777777",
                "#FBC15E",
                "#8EBA42",
                "#FFB5B8",
            ],
        )
    }
)

# References

In [ ]:
# Converting string column to datetime
df['Open'] = pd.to_datetime(df['OPEN TIME'],format= '%H:%M:%S' ).dt.time
df['Close'] = pd.to_datetime(df['CLOSE TIME'],format= '%H:%M:%S' ).dt.time

# Cleaning/removing one column or index row
df.drop('column_name', axis=1)
df.drop('row_name', axis=0)